In [1]:
import pandas as pd
import os
import json

alldicts = []
for file in os.listdir("data"):
    full_filename = "%s/%s" % ("data", file)
    with open(full_filename,'r') as fi:
        dict = json.load(fi)
        alldicts.append(dict)

In [2]:
df = pd.DataFrame.from_dict(alldicts)
df = df[["category","date_publish", "text", "title","description"]]

In [3]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df['desc']=df['description'].map(lambda s:preprocess(s)) 

In [26]:
df

,category,date_publish,text,title,description,desc
0,North Korea Missile,2017-11-06T22:18:21,TOKYO (Reuters) - U.S. President Donald Trump ...,Trump says Japan would shoot North Korean miss...,U.S. President Donald Trump said on Monday tha...,president donald trump said monday japan would...
1,Equifax breach,2017-11-14T10:01:09,When a credit card is lost or stolen—or if the...,How Much Will Equifax Pay?,A law that protects consumers’ data was writte...,law protects consumers data written age hacking
2,Dieselgate,2017-10-24T08:11:27,EU raids automaker BMW in post-Dieselgate cart...,EU raids automaker BMW in post-Dieselgate cart...,EU antitrust regulators have raided the office...,antitrust regulators raided offices automaker ...
3,Zimbabwe,2017-11-16T00:00:00,Zimbabwe's President Robert Mugabe was meeting...,Zimbabwe calm as Mugabe is urged to go peacefully,Zimbabwe's President Robert Mugabe was meeting...,zimbabwe president robert mugabe meeting south...
4,Zimbabwe,2017-11-17T00:00:00,“Do you think President Zuma would like to be ...,Zimbabwe human rights lawyer: Nothing is going...,Nothing is going to change in Zimbabwe‚ becaus...,nothing going change zimbabwe president robert...
5,Star Wars: Battlefront II - Loot Box,2017-11-17T01:39:18,"Star Wars: Battlefront II releases tomorrow, b...",Star Wars: Battlefront II Doing Away With Cont...,"Star Wars: Battlefront II releases tomorrow, b...",star wars battlefront releases tomorrow game e...
6,Dieselgate,2017-11-08T20:29:48,LONDON (Reuters) - Volkswagen (VOWG_p.DE) has ...,Volkswagen has still not fixed one in three di...,Volkswagen (VOWG_p.DE) has yet to fix around 3...,volkswagen vowg_p yet fix around million cars ...
7,IPhone-X Green-Line,2017-11-13T10:17:53,A SMALL number of iPhone X owners have claimed...,iPhone X owners report seeing 'green line of d...,A small number of users say an annoying line h...,small number users say annoying line popped sc...
8,North Korea Missile,2017-11-14T18:33:28,North Korea hasn’t fired a missile for 60 days...,North Korea's winter training means fewer miss...,North Korea hasn't fired a missile for 60 days...,north korea fired missile days may winter trai...
9,Star Wars: Battlefront II - Loot Box,2017-11-16T17:00:00,"It’s Thursday, and we’re almost at the end of ...",EA Respond To Star Wars Loot Box Gambling Inve...,Dragonball FighterZ’s roster grows with 3 new ...,dragonball fighterz roster grows new character...


In [4]:
import spacy
nlp = spacy.load('en_core_web_lg-2.1.0')
vectors = {}
docs = []

for d in df.desc:
    doc = nlp(d)
    docs.append(doc)
    vectors.update({d: doc.vector})

In [5]:
sen = list(vectors.keys())
vec = list(vectors.values())

In [6]:
import hdbscan
import numpy as np

x = np.array(vec)
n_topics = {}

for i in np.arange(0.001,1,0.002):
    dbscan = hdbscan(eps=i, min_samples=2, metric='cosine').fix(x)
    n_topics.update({i:len(pd.Series(dbscan.labels_).value_counts())})

/Users/Jenny/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/Jenny/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


TypeError: 'module' object is not callable